In [1]:
#!pip install opencv-python

In [2]:
# TRANSFER LEARNING FINE TUNING -  MASS CLASSIFICATION USING SEGMENTATION MODEL OUTPUT BCDR MASS LESION
import numpy as np
import tensorflow.keras
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
#%matplotlib inline
keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
#import pandas as pd
import os
#import matplotlib.pyplot as plt
#from tensorflow.python.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras import Sequential, Model

vid='v001_5' #last number is the cross-validation fold number

In [3]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.resnet_v2 import ResNet50V2
base_model=ResNet50V2(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(128)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 56, 56, 256)  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 56, 56, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_blo

 conv3_block2_2_conv (Conv2D)   (None, 28, 28, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 28, 28, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block1_1_conv (Conv2D)   (None, 14, 14, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block1_2_pad (ZeroPaddin  (None, 16, 16, 256)  0          ['conv4_block1_1_relu[0][0]']    
 g2D)     

                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 14, 14, 1024  263168      ['conv4_block3_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block3_out (Add)         (None, 14, 14, 1024  0           ['conv4_block2_out[0][0]',       
                                )                                 'conv4_block3_3_conv[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block6_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block6_2_pad (ZeroPaddin  (None, 16, 16, 256)  0          ['conv4_block6_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv4_block6_2_conv (Conv2D)   (None, 7, 7, 256)    589824      ['conv4_block6_2_pad[0][0]']     
          

                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 7, 7, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_out (Add)         (None, 7, 7, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_conv[0][0]']    
                                                                                                  
 conv5_block3_preact_bn (BatchN  (None, 7, 7, 2048)  8192        ['conv5_block2_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv5_block3_preact_relu (Acti  (None, 7, 7, 2048)  0           ['conv5_block3_preact_bn[0][0]'] 
 vation)  

In [5]:
len(model.layers)

197

In [6]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [7]:
# Image preprocessing and data augmentation
import cv2

batch_size=8

train_datagen = ImageDataGenerator(
                         horizontal_flip = True,
                         vertical_flip = True,
                         rotation_range = 90,
                         fill_mode="constant",
                         width_shift_range = 0.2,
                         zoom_range = 0.2,
                         rescale=1./255,
                         height_shift_range = 0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train0'+vid[-1], 
                                                  target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 184 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [8]:
#Callbacks
def get_callbacks(name_weights, patience_lr, patience):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=patience, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [9]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_class_resnet50v2_v001_"+vid[-1]+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10, patience=10) # do not reduce LR here

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=step_size_valid,
                    validation_data=valid_generator,
                    epochs=3,
                    verbose=1,
                    callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13776\3881320540.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
23/23 [==============================] - 18s 304ms/step - loss: 0.6301 - accuracy: 0.6957 - val_loss: 0.5784 - val_accuracy: 0.8250 - lr: 0.0010
Epoch 2/3
23/23 [==============================] - 5s 206ms/step - loss: 0.7064 - accuracy: 0.7446 - val_loss: 0.4460 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/3
23/23 [==============================] - 3s 131ms/step - loss: 0.5876 - accuracy: 0.7826 - val_loss: 0.6127 - val_accuracy: 0.7250 - lr: 0.0010


In [10]:
#Training stage 2
top_layer_num=round(len(model.layers)*0.75)

for layer in model.layers[:top_layer_num]:
    layer.trainable=False
for layer in model.layers[top_layer_num:]:  #last ~25% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10, patience=10) # do not reduce LR here

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=step_size_valid,
                    validation_data=valid_generator,
                    epochs=10,
                    verbose=1,
                    callbacks = callbacks)

Epoch 1/10


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13776\2189440975.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


23/23 [==============================] - 14s 324ms/step - loss: 0.5734 - accuracy: 0.7337 - val_loss: 0.4361 - val_accuracy: 0.8000 - lr: 1.0000e-04
Epoch 2/10
23/23 [==============================] - 3s 127ms/step - loss: 0.4651 - accuracy: 0.7880 - val_loss: 0.5105 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 3/10
23/23 [==============================] - 3s 131ms/step - loss: 0.4366 - accuracy: 0.8043 - val_loss: 0.4584 - val_accuracy: 0.8500 - lr: 1.0000e-04
Epoch 4/10
23/23 [==============================] - 5s 225ms/step - loss: 0.4464 - accuracy: 0.8261 - val_loss: 0.4042 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 5/10
23/23 [==============================] - 5s 239ms/step - loss: 0.4089 - accuracy: 0.8315 - val_loss: 0.3456 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 6/10
23/23 [==============================] - 3s 122ms/step - loss: 0.3677 - accuracy: 0.8478 - val_loss: 0.5453 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 7/10
23/23 [==============================] - 3s 1

In [11]:
#Training stage 3
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=5, patience=5)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=step_size_valid,
                    validation_data=valid_generator,
                    epochs=100,
                    verbose=1,
                    callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13776\943335175.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/100
23/23 [==============================] - 17s 373ms/step - loss: 0.4772 - accuracy: 0.7880 - val_loss: 0.3943 - val_accuracy: 0.9000 - lr: 1.0000e-05
Epoch 2/100
23/23 [==============================] - 3s 126ms/step - loss: 0.5122 - accuracy: 0.8043 - val_loss: 0.4074 - val_accuracy: 0.8750 - lr: 1.0000e-05
Epoch 3/100
23/23 [==============================] - 7s 295ms/step - loss: 0.3928 - accuracy: 0.7880 - val_loss: 0.3891 - val_accuracy: 0.8250 - lr: 1.0000e-05
Epoch 4/100
23/23 [==============================] - 6s 271ms/step - loss: 0.4154 - accuracy: 0.8043 - val_loss: 0.3296 - val_accuracy: 0.8500 - lr: 1.0000e-05
Epoch 5/100
23/23 [==============================] - 3s 121ms/step - loss: 0.4121 - accuracy: 0.7935 - val_loss: 0.3936 - val_accuracy: 0.8250 - lr: 1.0000e-05
Epoch 6/100
23/23 [==============================] - 3s 121ms/step - loss: 0.4161 - accuracy: 0.8152 - val_loss: 0.3753 - val_accuracy: 0.8500 - lr: 1.0000e-05
Epoch 7/100
23/23 [====================

In [12]:
import os, os.path

VDIR0 = 'valid0'+vid[-1]+'/ben'
VDIR1 = 'valid0'+vid[-1]+'/mal'

valid0=len([name for name in os.listdir(VDIR0) if os.path.isfile(os.path.join(VDIR0, name))])
valid1=len([name for name in os.listdir(VDIR1) if os.path.isfile(os.path.join(VDIR1, name))])

num_of_valid_samples=valid0+valid1
# Ref:  https://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python

In [13]:
# Validation accuracy and validation data confusion matrix

import sklearn
valid_generator = valid_datagen.flow_from_directory('valid0'+vid[-1], 
                                                     target_size=(IMG_SIZE,IMG_SIZE),
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)

Found 44 images belonging to 2 classes.
6/6 [==============================] - 2s 82ms/step
Confusion Matrix:
[[19  3]
 [ 4 18]]
Accuracy: 0.8409090909090909
ROC AUC score: 0.9132231404958678
Precision score: 0.8571428571428571
Recall score: 0.8181818181818182
F1 score: 0.8372093023255814
MCC score: 0.6825236327899351


In [14]:
# Predicted object delete threshold = 30
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('test_upp_xcept', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)

Found 42 images belonging to 2 classes.
6/6 [==============================] - 0s 53ms/step
Confusion Matrix:
[[24  1]
 [10  7]]
Accuracy: 0.7380952380952381
ROC AUC score: 0.7952941176470588
Precision score: 0.875
Recall score: 0.4117647058823529
F1 score: 0.56
MCC score: 0.4647058823529412
